In [1]:
from sklearn import svm


In [2]:
X=[[0,0],[1,1]]
Y=[0,1]

In [3]:
clf=svm.SVC(gamma='scale')

In [ ]:
clf.predict([[1.3, 0.1]])


In [8]:
from sklearn.datasets import fetch_20newsgroups

In [9]:
categories = ['alt.atheism','sci.med']


In [10]:
twenty_train = fetch_20newsgroups(subset='train',categories=categories, shuffle=True, random_state=42)


In [11]:
twenty_train.target_names

['alt.atheism', 'sci.med']

In [12]:
len(twenty_train.data)

1074

In [13]:
twenty_train.target[:10]

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 1])

In [14]:
for t in twenty_train.target:
    print(twenty_train.target_names[t])

alt.atheism
alt.atheism
alt.atheism
sci.med
alt.atheism
alt.atheism
alt.atheism
alt.atheism
alt.atheism
sci.med
alt.atheism
sci.med
sci.med
sci.med
sci.med
sci.med
sci.med
sci.med
sci.med
sci.med
alt.atheism
sci.med
sci.med
sci.med
sci.med
sci.med
sci.med
alt.atheism
alt.atheism
sci.med
alt.atheism
sci.med
sci.med
sci.med
alt.atheism
alt.atheism
sci.med
alt.atheism
sci.med
sci.med
alt.atheism
alt.atheism
sci.med
alt.atheism
sci.med
sci.med
alt.atheism
sci.med
alt.atheism
sci.med
sci.med
alt.atheism
sci.med
sci.med
alt.atheism
sci.med
alt.atheism
alt.atheism
alt.atheism
sci.med
alt.atheism
alt.atheism
alt.atheism
alt.atheism
alt.atheism
sci.med
sci.med
alt.atheism
sci.med
alt.atheism
sci.med
sci.med
sci.med
alt.atheism
sci.med
alt.atheism
alt.atheism
alt.atheism
sci.med
alt.atheism
sci.med
sci.med
alt.atheism
sci.med
sci.med
sci.med
sci.med
alt.atheism
sci.med
sci.med
alt.atheism
sci.med
alt.atheism
alt.atheism
alt.atheism
sci.med
sci.med
alt.atheism
sci.med
alt.atheism
sci.med
sci.med


In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)

In [17]:
X_train_counts.shape

(1074, 22735)

In [21]:
from sklearn.feature_extraction.text import TfidfTransformer

In [22]:
tf_transformer=TfidfTransformer(use_idf=False).fit(X_train_counts)

In [23]:
X_train_tf = tf_transformer.transform(X_train_counts)

In [24]:
X_train_tf.shape

(1074, 22735)

the feature is just occurance of each work in the text, now to train some type of classifyer...

In [26]:
from sklearn.naive_bayes import MultinomialNB

tfidf-the more the term appears in the document, the more its weight is. but its also standized by how many times the word appears in the ENTIRE dataset.
-moves stopwords

In [27]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [28]:
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [29]:
docs_new = ['Science and religion cannot go together','the surger was successful']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

In [30]:
predicted = clf.predict(X_new_tfidf)

In [31]:
type(predicted)

numpy.ndarray

In [33]:
for doc, category in zip(docs_new,predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'Science and religion cannot go together' => alt.atheism
'the surger was successful' => sci.med


In [34]:
from sklearn.pipeline import Pipeline

with a pipeline, you can do a lot of steps at once. here im counting word freq, transforming for weight, and putting it into a naive bayes.

In [36]:
text_clf = Pipeline([('vect',CountVectorizer()),
                    ('tfidf',TfidfTransformer()),
                    ('clf',MultinomialNB()),
                    ])

In [37]:
text_clf.fit(twenty_train.data,twenty_train.target)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [38]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test',categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)


In [39]:
np.mean(predicted==twenty_test.target)

0.965034965034965

In [40]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=5, tol=None)),])
text_clf.fit(twenty_train.data, twenty_train.target)  

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...dom_state=42, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))])

In [41]:
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target) 

0.9664335664335665

In [43]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,target_names=twenty_test.target_names))

              precision    recall  f1-score   support

 alt.atheism       0.98      0.94      0.96       319
     sci.med       0.96      0.98      0.97       396

   micro avg       0.97      0.97      0.97       715
   macro avg       0.97      0.96      0.97       715
weighted avg       0.97      0.97      0.97       715

